In [1]:
# Run this cell only when you are troubled with the following error especially if you are using conda to manage python environment
# `ImportError: libpython3.8.so.1.0: cannot open shared object file: No such file or directory
import os
from pathlib import Path
from ctypes import cdll
import sys

is_conda = 'CONDA_PREFIX' in os.environ or 'CONDA_DEFAULT_ENV' in os.environ
if is_conda:
    version_info = sys.version_info
    if version_info.major == 3 and version_info.minor >= 8:
        conda_lib_path = Path(
            sys.executable).parent.parent / f"lib/libpython{version_info.major}.{version_info.minor}.so.1.0"
    else:
        conda_lib_path = Path(
            sys.executable).parent.parent / f"lib/libpython{version_info.major}.{version_info.minor}m.so.1.0"
    python_lib = cdll.LoadLibrary(str(conda_lib_path))
    print(f"Load Python lib {conda_lib_path}")

Load Python lib /home/azureuser/miniconda3/envs/rlgpu/lib/libpython3.8.so.1.0


In [ ]:
import furniture_bench
import gym

import torch

: 

In [3]:
import os
from datetime import datetime
import numpy as np

# noinspection PyUnresolvedReferences
import isaacgym
from sim_web_visualizer.isaac_visualizer_client import create_isaac_visualizer, bind_visualizer_to_gym, set_gpu_pipeline
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess

sys.path.append(os.path.join(os.path.abspath(''), 'IsaacGymEnvs'))

import hydra

from isaacgymenvs.pbt.pbt import PbtAlgoObserver, initial_pbt_check
from hydra.utils import to_absolute_path
from isaacgymenvs.tasks import isaacgym_task_map
from omegaconf import DictConfig, OmegaConf
import gym

from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed

/home/azureuser/miniconda3/envs/rlgpu/lib/python3.8/site-packages/meshcat/servers/zmqserver.py:18: DeprecationWarning: zmq.eventloop.ioloop is deprecated in pyzmq 17. pyzmq now works with default tornado and asyncio eventloops.
  import zmq.eventloop.ioloop
/home/azureuser/sim-web-visualizer/sim_web_visualizer/__init__.py:15: UserWarning: No module named 'sapien'
  warnings.warn(str(e))
/home/azureuser/sim-web-visualizer/sim_web_visualizer/__init__.py:16: UserWarning: 
No Sapien python library installed. Disable Sapien Visualizer.
 If you want to Sapien Visualizer, please consider install it via: pip3 install sapien
  warnings.warn(


PyTorch version 2.0.1+cu117
Device count 1
/home/azureuser/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/azureuser/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/azureuser/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...
2023-09-10 19:56:47,301 - INFO - logger - logger initialized
/home/azureuser/IsaacGymEnvs/isaacgymenvs/tasks/amp/poselib/poselib/core/rotation3d.py:222: DeprecationWarning: invalid escape sequence \*
  """


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


In [4]:
# This is equivalent to run `meshcat-server` inside a separate terminal window
start_zmq_server_as_subprocess()

################################################################################################
# The following code is only used for visualizer and not presented in the original IsaacGymEnv
################################################################################################
from isaacgymenvs.tasks.base import vec_task
from isaacgym import gymapi


def wrapped_create_sim(self: vec_task.VecTask, compute_device: int, graphics_device: int, physics_engine,
                       sim_params: gymapi.SimParams):
    sim = vec_task._create_sim_once(self.gym, compute_device, graphics_device, physics_engine, sim_params)
    if sim is None:
        print("*** Failed to create sim")
        quit()
    self.gym = bind_visualizer_to_gym(self.gym, sim)
    set_gpu_pipeline(sim_params.use_gpu_pipeline)
    return sim


# Reload VecTask function to create a hook for sim_web_visualizer
vec_task.VecTask.create_sim = wrapped_create_sim

# Create web visualizer
web_visualizer = create_isaac_visualizer(port=6000, host="localhost", keep_default_viewer=False, max_env=16,
                                         scene_offset=np.array([40.0, 40.0]))


################################################################################################
# End of visualizer code
################################################################################################

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Deleting all previous scene asset.
For efficiency, the sim_web_visualizer will only visualize the first 16 environments.
Modify the max_env parameter if you prefer more or less visualization.
Deleting all previous scene asset.


In [5]:
################################################################################################
# The following code is the same as the original IsaacGymEnv
# https://github.com/NVIDIA-Omniverse/IsaacGymEnvs/blob/main/isaacgymenvs/train.py
################################################################################################

def preprocess_train_config(cfg, config_dict):
    """
    Adding common configuration parameters to the rl_games train config.
    An alternative to this is inferring them in task-specific .yaml files, but that requires repeating the same
    variable interpolations in each config.
    """

    train_cfg = config_dict['params']['config']

    train_cfg['device'] = cfg.rl_device

    train_cfg['population_based_training'] = cfg.pbt.enabled
    train_cfg['pbt_idx'] = cfg.pbt.policy_idx if cfg.pbt.enabled else None

    train_cfg['full_experiment_name'] = cfg.get('full_experiment_name')

    print(f'Using rl_device: {cfg.rl_device}')
    print(f'Using sim_device: {cfg.sim_device}')
    print(train_cfg)

    try:
        model_size_multiplier = config_dict['params']['network']['mlp']['model_size_multiplier']
        if model_size_multiplier != 1:
            units = config_dict['params']['network']['mlp']['units']
            for i, u in enumerate(units):
                units[i] = u * model_size_multiplier
            print(
                f'Modified MLP units by x{model_size_multiplier} to {config_dict["params"]["network"]["mlp"]["units"]}')
    except KeyError:
        pass

    return config_dict


def launch_rlg_hydra(cfg: DictConfig):
    if cfg.pbt.enabled:
        initial_pbt_check(cfg)

    from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, MultiObserver, ComplexObsRLGPUEnv
    from isaacgymenvs.utils.wandb_utils import WandbAlgoObserver
    from rl_games.common import env_configurations, vecenv
    from rl_games.torch_runner import Runner
    from rl_games.algos_torch import model_builder
    from isaacgymenvs.learning import amp_continuous
    from isaacgymenvs.learning import amp_players
    from isaacgymenvs.learning import amp_models
    from isaacgymenvs.learning import amp_network_builder
    import isaacgymenvs

    time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_name = f"{cfg.wandb_name}_{time_str}"

    # ensure checkpoints can be specified as relative paths
    if cfg.checkpoint:
        cfg.checkpoint = to_absolute_path(cfg.checkpoint)

    cfg_dict = omegaconf_to_dict(cfg)
    print_dict(cfg_dict)

    # set numpy formatting for printing only
    set_np_formatting()

    # global rank of the GPU
    global_rank = int(os.getenv("RANK", "0"))

    # sets seed. if seed is -1 will pick a random one
    cfg.seed = set_seed(cfg.seed, torch_deterministic=cfg.torch_deterministic, rank=global_rank)

    def create_isaacgym_env(**kwargs):
        envs = isaacgymenvs.make(
            cfg.seed,
            cfg.task_name,
            cfg.task.env.numEnvs,
            cfg.sim_device,
            cfg.rl_device,
            cfg.graphics_device_id,
            cfg.headless,
            cfg.multi_gpu,
            cfg.capture_video,
            cfg.force_render,
            cfg,
            **kwargs,
        )
        if cfg.capture_video:
            envs.is_vector_env = True
            envs = gym.wrappers.RecordVideo(
                envs,
                f"videos/{run_name}",
                step_trigger=lambda step: step % cfg.capture_video_freq == 0,
                video_length=cfg.capture_video_len,
            )
        return envs

    env_configurations.register('rlgpu', {
        'vecenv_type': 'RLGPU',
        'env_creator': lambda **kwargs: create_isaacgym_env(**kwargs),
    })

    ige_env_cls = isaacgym_task_map[cfg.task_name]
    dict_cls = ige_env_cls.dict_obs_cls if hasattr(ige_env_cls, 'dict_obs_cls') and ige_env_cls.dict_obs_cls else False

    if dict_cls:

        obs_spec = {}
        actor_net_cfg = cfg.train.params.network
        obs_spec['obs'] = {'names': list(actor_net_cfg.inputs.keys()),
                           'concat': not actor_net_cfg.name == "complex_net", 'space_name': 'observation_space'}
        if "central_value_config" in cfg.train.params.config:
            critic_net_cfg = cfg.train.params.config.central_value_config.network
            obs_spec['states'] = {'names': list(critic_net_cfg.inputs.keys()),
                                  'concat': not critic_net_cfg.name == "complex_net", 'space_name': 'state_space'}

        vecenv.register('RLGPU',
                        lambda config_name, num_actors, **kwargs: ComplexObsRLGPUEnv(config_name, num_actors, obs_spec,
                                                                                     **kwargs))
    else:

        vecenv.register('RLGPU', lambda config_name, num_actors, **kwargs: RLGPUEnv(config_name, num_actors, **kwargs))

    rlg_config_dict = omegaconf_to_dict(cfg.train)
    rlg_config_dict = preprocess_train_config(cfg, rlg_config_dict)

    observers = [RLGPUAlgoObserver()]

    if cfg.pbt.enabled:
        pbt_observer = PbtAlgoObserver(cfg)
        observers.append(pbt_observer)

    if cfg.wandb_activate:
        cfg.seed += global_rank
        if global_rank == 0:
            # initialize wandb only once per multi-gpu run
            wandb_observer = WandbAlgoObserver(cfg)
            observers.append(wandb_observer)

    # register new AMP network builder and agent
    def build_runner(algo_observer):
        runner = Runner(algo_observer)
        runner.algo_factory.register_builder('amp_continuous', lambda **kwargs: amp_continuous.AMPAgent(**kwargs))
        runner.player_factory.register_builder('amp_continuous',
                                               lambda **kwargs: amp_players.AMPPlayerContinuous(**kwargs))
        model_builder.register_model('continuous_amp', lambda network, **kwargs: amp_models.ModelAMPContinuous(network))
        model_builder.register_network('amp', lambda **kwargs: amp_network_builder.AMPBuilder())

        return runner

    # convert CLI arguments into dictionary
    # create runner and set the settings
    runner = build_runner(MultiObserver(observers))
    runner.load(rlg_config_dict)
    runner.reset()

    # dump config dict
    if not cfg.test:
        experiment_dir = os.path.join('runs', cfg.train.params.config.name +
                                      '_{date:%d-%H-%M-%S}'.format(date=datetime.now()))

        os.makedirs(experiment_dir, exist_ok=True)
        with open(os.path.join(experiment_dir, 'config.yaml'), 'w') as f:
            f.write(OmegaConf.to_yaml(cfg))

    runner.run({
        'train': not cfg.test,
        'play': cfg.test,
        'checkpoint': cfg.checkpoint,
        'sigma': cfg.sigma if cfg.sigma != '' else None
    })

In [6]:
# Comment out the hydra/job_logging in the config to avoid a hydra error.
# This is nothing related to the web visualizer but will raise a multiple value error without the following modification
# You do not need to do that if you are not using hydra compose API.
!sed -i 's/- hydra/#- hydra/g' IsaacGymEnvs/isaacgymenvs/cfg/config.yaml

sed: can't read IsaacGymEnvs/isaacgymenvs/cfg/config.yaml: No such file or directory


In [9]:
# Modify the IP if the jupyter server is not localhost
# If your remote server has a public ip, the machine ip should be that one
# If you are using ssh port forwarding like: ssh YOUR_SERVER -L 7000:localhost:7000 to forward port 7000 from your server to local machine, it should still be your localhost
machine_ip = "127.0.0.1"
web_visualizer.jupyter_cell(url=f"http://{machine_ip}:7000/static/", height=400)

In [11]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
config_path = "../IsaacGymEnvs/isaacgymenvs/cfg"
task_name = "Ant"
with hydra.initialize(version_base=None, config_path=config_path):
    cfg = hydra.compose(config_name='config.yaml', overrides=[f"task={task_name}"])
launch_rlg_hydra(cfg)

/home/azureuser/miniconda3/envs/rlgpu/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/azureuser/miniconda3/envs/rlgpu/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/azureuser/miniconda3/envs/rlgpu/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


task: 
    name: Ant
    physics_engine: physx
    env: 
        numEnvs: 4096
        envSpacing: 5
        episodeLength: 1000
        enableDebugVis: False
        clipActions: 1.0
        powerScale: 1.0
        controlFrequencyInv: 1
        headingWeight: 0.5
        upWeight: 0.1
        actionsCost: 0.005
        energyCost: 0.05
        dofVelocityScale: 0.2
        contactForceScale: 0.1
        jointsAtLimitCost: 0.1
        deathCost: -2.0
        terminationHeight: 0.31
        plane: 
            staticFriction: 1.0
            dynamicFriction: 1.0
            restitution: 0.0
        asset: 
            assetFileName: mjcf/nv_ant.xml
        enableCameraSensors: False
    sim: 
        dt: 0.0166
        substeps: 2
        up_axis: z
        use_gpu_pipeline: True
        gravity: [0.0, 0.0, -9.81]
        physx: 
            num_threads: 4
            solver_type: 1
            use_gpu: True
            num_position_iterations: 4
            num_velocity_iterations: 0


/home/azureuser/IsaacGymEnvs/isaacgymenvs/tasks/ant.py:151: DeprecationWarning: an integer is required (got type isaacgym._bindings.linux-x86_64.gym_38.DofDriveMode).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  asset_options.default_dof_drive_mode = gymapi.DOF_MODE_NONE
/home/azureuser/miniconda3/envs/rlgpu/lib/python3.8/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


Box(-1.0, 1.0, (8,), float32) Box(-inf, inf, (60,), float32)
current training device: cuda:0
build mlp: 60
RunningMeanStd:  (1,)
RunningMeanStd:  (60,)
Setup tensor for GPU pipeline.
fps step: 16739 fps step and policy inference: 14573 fps total: 13652 epoch: 1/500 frames: 0
fps step: 35074 fps step and policy inference: 34344 fps total: 33401 epoch: 2/500 frames: 65536
fps step: 32262 fps step and policy inference: 31630 fps total: 30856 epoch: 3/500 frames: 131072
fps step: 34468 fps step and policy inference: 33645 fps total: 32756 epoch: 4/500 frames: 196608
fps step: 33744 fps step and policy inference: 32894 fps total: 32079 epoch: 5/500 frames: 262144
fps step: 35164 fps step and policy inference: 34327 fps total: 33442 epoch: 6/500 frames: 327680
fps step: 34836 fps step and policy inference: 34007 fps total: 33139 epoch: 7/500 frames: 393216
fps step: 34812 fps step and policy inference: 33963 fps total: 33106 epoch: 8/500 frames: 458752
fps step: 35287 fps step and policy inf

KeyboardInterrupt: 

Bad pipe message: %s [b'\x12\x8d)]*e\xd49w\xcf|7"K\xc0\xb0e\xa4 \x81\xbfW\x98\xc8\xbeAwX\xa1.\xa5S\xa8D0\xb5\xd2\t\x07\xbc<\xd6\xac\xfc0\x14\xf4\xef\xb6 g\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00']
Bad pipe message: %s [b'#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03']
Bad pipe message: %s [b'\x08\x08\x08\t\x08\n\x08', b'\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06']
Bad pipe message: %s [b'\xeb\xbe\xfb\xf9\x87C\x9c\xeeB\xcf\xe8\xf3\x97\x03\xcc\x02\x1de \xf1\xb9>\xf2\xd6\xe40G\x95\n\xb3\x9c\xfcM`\x84\xe9\x87u\xde\xb5\xaf\xb0\xb3N\x9c;dm\x14\xda\x01\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c', b

In [2]:
env = gym.make(
  "FurnitureSimState-v0",
  furniture="lamp",         # Specifies the type of furniture [lamp | square_table | desk | drawer | cabinet | round_table | stool | chair | one_leg].
  num_envs=1,               # Number of parallel environments.
  resize_img=True,          # If true, images are resized to 224 x 224.
  headless=True,            # If true, simulation runs without GUI.
  compute_device_id=0,      # GPU device ID for simulation.
  graphics_device_id=0,     # GPU device ID for rendering.
  init_assembled=False,     # If true, the environment is initialized with assembled furniture.
  np_step_out=False,        # If true, env.step() returns Numpy arrays.
  channel_first=False,      # If true, images are returned in channel first format.
  randomness="low",         # Level of randomness in the environment [low | med | high].
  high_random_idx=-1,       # Index of the high randomness level (range: [0-2]). Default -1 will randomly select the index within the range.
  save_camera_input=False,  # If true, the initial camera inputs are saved.
  record=True,              # If true, videos of the wrist and front cameras' RGB inputs are recorded.
  max_env_steps=3000,       # Maximum number of steps per episode.
  act_rot_repr='quat'       # Representation of rotation for action space. Options are 'quat' and 'axis'.
)

PyTorch version 2.0.1+cu117
Device count 1
/home/azureuser/isaacgym/python/isaacgym/_bindings/src/gymtorch


/home/azureuser/miniconda3/envs/rlgpu/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Using /home/azureuser/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/azureuser/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


: 

: 

In [3]:
env.reset()

INFO: Found collision-free init pose


ValueError: need at least one array to concatenate

In [4]:
done = False

while not done:
    ac = torch.tensor(env.action_space.sample()).float().to('cuda') # (1, 8) torch.Tensor
    ob, rew, done, _ = env.step(ac)

tensor([[False]], device='cuda:0')